# Conjoints with LLMs

In [1]:
import itertools
import random
import pandas as pd

from lludens.agent import TotreLLM

In [2]:
# 1. Define Attributes and Levels
attributes = {
    "bag_color": ["red", "blue"],
    "flavor": ["sour cream & onion", "bbq"],
    "ruffled": ["yes", "no"],
}

# 2. Generate All Possible Combinations (Cartesian Product)
combinations = list(itertools.product(*attributes.values()))


# 3. Create a Function to Format a Choice for the LLM
def format_choice(combination):
    description = []
    for i, attr_name in enumerate(attributes.keys()):
        description.append(f"{attr_name.replace('_', ' ').title()}: {combination[i]}")
    return "; ".join(description)


# 4. Define the System Prompt for the LLM
system_prompt = """
You are a consumer participating in a survey about potato chips.
You will be presented with two options for bags of chips, described by their attributes.
Choose the option you would be more likely to purchase. Respond with either 'A' or 'B'.
Be consistent in your preferences.
""".strip()

In [3]:
# 5. Create the LLM Agent
agent = TotreLLM(model_id="4o", system=system_prompt, options={"temperature": 0.1})

pairwise_combinations = list(itertools.combinations(combinations, 2))
random.shuffle(pairwise_combinations)  # Shuffle the pairwise combinations
results = []
for choice1, choice2 in pairwise_combinations:
    prompt = (
        "Choose between the following two options:\n"
        f"A: {format_choice(choice1)}\n"
        f"B: {format_choice(choice2)}\n"
        "Your choice (A or B): "
    )

    response = agent.interact(prompt)
    # Basic response validation (you might need more robust handling)
    if response.lower() not in ["a", "b"]:
        print(f"Warning: Invalid response '{response}'. Skipping.")
        chosen_combination = "invalid"
        # continue

    if response.lower() == "a":
        chosen_combination = choice1
    elif response.lower() == "b":
        chosen_combination = choice2

    results.append(
        {
            "bag_color_A": choice1[0],
            "flavor_A": choice1[1],
            "ruffled_A": choice1[2],
            "bag_color_B": choice2[0],
            "flavor_B": choice2[1],
            "ruffled_B": choice2[2],
            "choice": response.lower(),
            "chosen_combination": chosen_combination,
        }
    )

# 7. Store Results in a Pandas DataFrame
df_results = pd.DataFrame(results)
df_results

,bag_color_A,flavor_A,ruffled_A,bag_color_B,flavor_B,ruffled_B,choice,chosen_combination
0,red,sour cream & onion,no,blue,sour cream & onion,yes,b,"(blue, sour cream & onion, yes)"
1,blue,sour cream & onion,yes,blue,bbq,yes,a,"(blue, sour cream & onion, yes)"
2,red,sour cream & onion,yes,blue,bbq,yes,a,"(red, sour cream & onion, yes)"
3,blue,sour cream & onion,yes,blue,bbq,no,a,"(blue, sour cream & onion, yes)"
4,blue,bbq,yes,blue,bbq,no,a,"(blue, bbq, yes)"
5,red,sour cream & onion,yes,red,bbq,yes,a,"(red, sour cream & onion, yes)"
6,red,bbq,yes,blue,sour cream & onion,no,b,"(blue, sour cream & onion, no)"
7,red,bbq,yes,blue,sour cream & onion,yes,b,"(blue, sour cream & onion, yes)"
8,red,bbq,yes,blue,bbq,yes,b,"(blue, bbq, yes)"
9,red,bbq,yes,blue,bbq,no,a,"(red, bbq, yes)"


In [4]:
# 8. Analyze Results (Example: Simple Preference Counts)
print("\n--- Overall Choice Counts ---")
print(df_results["choice"].value_counts())

print("\n--- Chosen Combination Counts ---")
print(df_results["chosen_combination"].value_counts())


--- Overall Choice Counts ---
choice
a    16
b    12
Name: count, dtype: int64

--- Chosen Combination Counts ---
chosen_combination
(blue, sour cream & onion, yes)    7
(red, sour cream & onion, yes)     6
(blue, sour cream & onion, no)     5
(blue, bbq, yes)                   4
(red, bbq, yes)                    3
(red, sour cream & onion, no)      2
(blue, bbq, no)                    1
Name: count, dtype: int64


In [5]:
def analyze_preferences(df, attribute):
    print(f"\n--- Preferences by {attribute.title()} ---")
    if "chosen_combination" in df.columns:
        for level in attributes[attribute]:
            count = (
                df["chosen_combination"]
                .apply(
                    lambda x: isinstance(x, tuple)
                    and x[list(attributes.keys()).index(attribute)] == level
                )
                .sum()
            )
            print(f"{level.title()}: {count}")
    else:
        print("chosen_combination column is missing in dataframe")


for attribute in attributes:
    analyze_preferences(df_results, attribute)


--- Preferences by Bag_Color ---
Red: 11
Blue: 17

--- Preferences by Flavor ---
Sour Cream & Onion: 20
Bbq: 8

--- Preferences by Ruffled ---
Yes: 20
No: 8


In [6]:
print("\n--- Introspection ---")
print(
    agent.interact(
        "Reflect on your choices. Which attribute was most important to you, and why?"
    )
)
print(agent.interact("Did you notice any patterns in your preferences?"))


--- Introspection ---
The most important attribute in my choices was whether the chips were ruffled or not. I consistently preferred ruffled chips, as they often provide a different texture and can enhance the overall eating experience. Additionally, I showed a preference for sour cream & onion flavor over bbq, but the presence of ruffles was a more decisive factor in my choices.
Yes, there were a few patterns in my preferences. I consistently preferred ruffled chips over non-ruffled ones, as the texture they provide is appealing. Additionally, I showed a preference for sour cream & onion flavor over bbq when both options were available with the same ruffle attribute. However, the presence of ruffles was the most decisive factor in my choices.


### It remembers everything

In [7]:
agent.conversation.responses

[<Response prompt='Choose between the following two options:
 A: Bag Color: red; Flavor: sour cream & onion; Ruffled: no
 B: Bag Color: blue; Flavor: sour cream & onion; Ruffled: yes
 Your choice (A or B): ' text='B'>,
 <Response prompt='Choose between the following two options:
 A: Bag Color: blue; Flavor: sour cream & onion; Ruffled: yes
 B: Bag Color: blue; Flavor: bbq; Ruffled: yes
 Your choice (A or B): ' text='A'>,
 <Response prompt='Choose between the following two options:
 A: Bag Color: red; Flavor: sour cream & onion; Ruffled: yes
 B: Bag Color: blue; Flavor: bbq; Ruffled: yes
 Your choice (A or B): ' text='A'>,
 <Response prompt='Choose between the following two options:
 A: Bag Color: blue; Flavor: sour cream & onion; Ruffled: yes
 B: Bag Color: blue; Flavor: bbq; Ruffled: no
 Your choice (A or B): ' text='A'>,
 <Response prompt='Choose between the following two options:
 A: Bag Color: blue; Flavor: bbq; Ruffled: yes
 B: Bag Color: blue; Flavor: bbq; Ruffled: no
 Your choic